TFIDF

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import nltk
nltk.download('stopwords') 
import re 
from bs4 import BeautifulSoup
import numpy as np
import torch

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
!pip install transformers
import transformers as ppb 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.8 MB/s eta 0:00:00


In [4]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/data.csv"
df = pd.read_csv(path)
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [8]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
#define bins
bins = [0,2,3,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [10]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [12]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [13]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')

In [14]:
def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split()if word not in STOPWORDS) # delete stopwors from text
    return text

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [16]:
df.iloc[:,0] = df.iloc[:,0].apply(clean_text)
print(df.iloc[:100,0])

0     constants alpha beta frac x alpha x+ beta frac...
1     length segment points 2a a4 4 1 2 sqrt 10 unit...
2     chewbacca 20 pieces cherry gum 30 pieces grape...
3     constants b let f x left begin array cl ax + b...
4             calculate sum 1 + 3 + 5 + cdots + 15 + 17
                            ...                        
95    largest value x satisfies equation sqrt 2x 4x ...
96    sum left dfrac 1 3 right + left dfrac 1 3 righ...
97    map 12centimeter length represents 72 kilomete...
98                      find sum values x 2 x23x2 4 x 4
99              given 0 f g 8 f x x2 + 8 g x x2 4 value
Name: Problem , Length: 100, dtype: object


In [17]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,constants alpha beta frac x alpha x+ beta frac...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,length segment points 2a a4 4 1 2 sqrt 10 unit...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,chewbacca 20 pieces cherry gum 30 pieces grape...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,constants b let f x left begin array cl ax + b...,3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,calculate sum 1 + 3 + 5 + cdots + 15 + 17,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [18]:
df.size

10464

In [19]:
df.shape

(1744, 6)

In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=word_tokenize,ngram_range=(1,2), binary=True, max_features=765)
TFIDF=vect.fit_transform(df['Problem '])

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    min_df=1,  # min count for relevant vocabulary
    max_features=765,  # maximum number of features
    strip_accents='unicode',  # replace all accented unicode char 
    # by their corresponding  ASCII char
    analyzer='word',  # features made of words
    token_pattern=r'\w{1,}',  # tokenize only words of 4+ chars
    ngram_range=(1, 1),  # features made of a single tokens
    use_idf=True,  # enable inverse-document-frequency reweighting
    smooth_idf=True,  # prevents zero division for unseen words
    sublinear_tf=False)

tfidf_df = tfidf_vectorizer.fit_transform(df['Problem '])

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
v = TfidfVectorizer()
vect = TfidfVectorizer(tokenizer=word_tokenize,ngram_range=(1,2), binary=True, max_features=765)
x = v.fit_transform(df['Problem ']) 

In [24]:
tfidf_vectorizer.vocabulary_

{'constants': 225,
 'alpha': 130,
 'beta': 166,
 'frac': 330,
 'x': 721,
 'x2': 724,
 'length': 421,
 'segment': 590,
 'points': 530,
 '2a': 49,
 '4': 79,
 '1': 11,
 '2': 33,
 'sqrt': 620,
 '10': 12,
 'units': 679,
 'product': 542,
 'possible': 535,
 'values': 691,
 '20': 34,
 'pieces': 525,
 'gum': 360,
 '30': 60,
 'complete': 214,
 'exactly': 297,
 'one': 489,
 'ratio': 557,
 'number': 483,
 '5': 89,
 'find': 318,
 'b': 154,
 'let': 425,
 'f': 308,
 'left': 418,
 'begin': 165,
 'array': 141,
 'cl': 204,
 'ax': 148,
 'text': 646,
 '8': 105,
 '3x': 76,
 'ge': 341,
 'end': 284,
 'right': 573,
 'function': 337,
 'calculate': 187,
 'sum': 631,
 '3': 59,
 'cdots': 195,
 '15': 24,
 '17': 29,
 'area': 138,
 'circle': 202,
 'center': 197,
 'p': 498,
 'q': 548,
 'express': 303,
 'answer': 136,
 'terms': 644,
 'pi': 519,
 'asy': 145,
 'size': 606,
 '150': 25,
 'pair': 500,
 '9': 110,
 'string': 628,
 'return': 571,
 'draw': 272,
 '0': 0,
 'arrows': 142,
 'dot': 267,
 'se': 587,
 'linewidth': 43

In [25]:
df.shape

(1744, 6)

In [26]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,constants alpha beta frac x alpha x+ beta frac...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,length segment points 2a a4 4 1 2 sqrt 10 unit...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,chewbacca 20 pieces cherry gum 30 pieces grape...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,constants b let f x left begin array cl ax + b...,3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,calculate sum 1 + 3 + 5 + cdots + 15 + 17,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [27]:
x = tfidf.fit_transform(df.iloc[:,0])
df_tfidf = pd.DataFrame(x.toarray())
x=np.array(df_tfidf)

In [28]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
x.size

6107488

In [30]:
from google.colab import files
df_tfidf.to_csv('tfidf.csv') 
files.download('tfidf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/tfidf.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,3492,3493,3494,3495,3496,3497,3498,3499,3500,3501
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
x.shape

(1744, 3503)

In [33]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,3492,3493,3494,3495,3496,3497,3498,3499,3500,3501
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
x['Level'] = df['Level '].values
x.head()

,0,1,2,3,4,5,6,7,8,9,...,3493,3494,3495,3496,3497,3498,3499,3500,3501,Level
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [35]:
x.drop(x.columns[[-1]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,3492,3493,3494,3495,3496,3497,3498,3499,3500,3501
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
y=df['Level ']

In [37]:
y.head()

0    3
1    3
2    3
3    3
4    1
Name: Level , dtype: int64

**Train Test Split**

In [38]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 3502), (524, 3502))

In [39]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

3    48.196721
1    29.590164
2    22.213115
Name: Level , dtype: float64


3    46.946565
1    29.961832
2    23.091603
Name: Level , dtype: float64


**Model Building**

In [40]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [41]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test)

Perceptron F1 score: 0.549330983124481


**Decision Tree**

In [42]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [43]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [44]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [45]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [46]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [47]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.491803,0.491803,0.557982,0.464107,0.45229,0.45229,0.496763,0.411747


**Random Forest**

In [48]:
clf_rf = RandomForestClassifier(class_weight="balanced")

In [49]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [50]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [51]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    0.9917    0.9958       361
           2     0.9891    1.0000    0.9945       271
           3     1.0000    1.0000    1.0000       588

    accuracy                         0.9975      1220
   macro avg     0.9964    0.9972    0.9968      1220
weighted avg     0.9976    0.9975    0.9975      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5519    0.7452    0.6341       157
           2     0.4857    0.1405    0.2179       121
           3     0.6643    0.7480    0.7036       246

    accuracy                         0.6069       524
   macro avg     0.5673    0.5446    0.5186       524
weighted avg     0.5894    0.6069    0.5707       524



In [52]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.491803,0.491803,0.557982,0.464107,0.45229,0.45229,0.496763,0.411747
1,RandomForest,0.997541,0.997541,0.997568,0.997543,0.60687,0.60687,0.589362,0.570661


**AdaBoost**

In [53]:
clf_adaboost =  AdaBoostClassifier()

In [54]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [55]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [56]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6831    0.4598    0.5497       361
           2     0.6104    0.1734    0.2701       271
           3     0.5900    0.9031    0.7137       588

    accuracy                         0.6098      1220
   macro avg     0.6278    0.5121    0.5112      1220
weighted avg     0.6221    0.6098    0.5666      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5766    0.4076    0.4776       157
           2     0.2581    0.0661    0.1053       121
           3     0.5366    0.8333    0.6529       246

    accuracy                         0.5286       524
   macro avg     0.4571    0.4357    0.4119       524
weighted avg     0.4843    0.5286    0.4739       524



In [57]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.491803,0.491803,0.557982,0.464107,0.452290,0.452290,0.496763,0.411747
1,RandomForest,0.997541,0.997541,0.997568,0.997543,0.606870,0.606870,0.589362,0.570661
2,Adaboost,0.609836,0.609836,0.622086,0.566634,0.528626,0.528626,0.484283,0.473907


**Gradient Boosting Classifier**

In [58]:
clf_gbm = GradientBoostingClassifier()

In [59]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [60]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [61]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.8626    0.7479    0.8012       361
           2     0.9840    0.4539    0.6212       271
           3     0.7379    0.9813    0.8423       588

    accuracy                         0.7951      1220
   macro avg     0.8615    0.7277    0.7549      1220
weighted avg     0.8294    0.7951    0.7810      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5379    0.4522    0.4913       157
           2     0.2812    0.0744    0.1176       121
           3     0.5750    0.8415    0.6832       246

    accuracy                         0.5477       524
   macro avg     0.4647    0.4560    0.4307       524
weighted avg     0.4960    0.5477    0.4951       524



In [62]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.491803,0.491803,0.557982,0.464107,0.452290,0.452290,0.496763,0.411747
1,RandomForest,0.997541,0.997541,0.997568,0.997543,0.606870,0.606870,0.589362,0.570661
2,Adaboost,0.609836,0.609836,0.622086,0.566634,0.528626,0.528626,0.484283,0.473907
3,GBM,0.795082,0.795082,0.829448,0.781041,0.547710,0.547710,0.496046,0.495108


**SVM Model**

In [63]:
linear_svm = SVC(kernel='linear', C=1, class_weight="balanced")

In [64]:
linear_svm.fit(X=X_train, y= y_train) 

SVC(C=1, class_weight='balanced', kernel='linear')

In [65]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [66]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.8136    0.8947    0.8522       361
           2     0.7457    0.8007    0.7722       271
           3     0.9117    0.8248    0.8661       588

    accuracy                         0.8402      1220
   macro avg     0.8237    0.8401    0.8302      1220
weighted avg     0.8458    0.8402    0.8411      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5169    0.6815    0.5879       157
           2     0.3559    0.3471    0.3515       121
           3     0.6935    0.5610    0.6202       246

    accuracy                         0.5477       524
   macro avg     0.5221    0.5299    0.5199       524
weighted avg     0.5626    0.5477    0.5485       524



In [67]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.491803,0.491803,0.557982,0.464107,0.452290,0.452290,0.496763,0.411747
1,RandomForest,0.997541,0.997541,0.997568,0.997543,0.606870,0.606870,0.589362,0.570661
2,Adaboost,0.609836,0.609836,0.622086,0.566634,0.528626,0.528626,0.484283,0.473907
3,GBM,0.795082,0.795082,0.829448,0.781041,0.547710,0.547710,0.496046,0.495108
4,SVM,0.840164,0.840164,0.845778,0.841137,0.547710,0.547710,0.562625,0.548482


**KNN**

In [68]:
knn = KNeighborsClassifier(n_neighbors=7)

In [69]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [70]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [71]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5566    0.6814    0.6127       361
           2     0.5125    0.3026    0.3805       271
           3     0.7120    0.7483    0.7297       588

    accuracy                         0.6295      1220
   macro avg     0.5937    0.5774    0.5743      1220
weighted avg     0.6217    0.6295    0.6175      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.4460    0.6051    0.5135       157
           2     0.3016    0.1570    0.2065       121
           3     0.6048    0.6098    0.6073       246

    accuracy                         0.5038       524
   macro avg     0.4508    0.4573    0.4424       524
weighted avg     0.4872    0.5038    0.4866       524



In [72]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.491803,0.491803,0.557982,0.464107,0.452290,0.452290,0.496763,0.411747
1,RandomForest,0.997541,0.997541,0.997568,0.997543,0.606870,0.606870,0.589362,0.570661
2,Adaboost,0.609836,0.609836,0.622086,0.566634,0.528626,0.528626,0.484283,0.473907
3,GBM,0.795082,0.795082,0.829448,0.781041,0.547710,0.547710,0.496046,0.495108
4,SVM,0.840164,0.840164,0.845778,0.841137,0.547710,0.547710,0.562625,0.548482
5,KNN,0.629508,0.629508,0.621678,0.617507,0.503817,0.503817,0.487225,0.486648


**GNB**

In [73]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [74]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [75]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [76]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9187    0.7202    0.8075       361
           2     0.5766    1.0000    0.7314       271
           3     1.0000    0.7942    0.8853       588

    accuracy                         0.8180      1220
   macro avg     0.8318    0.8381    0.8081      1220
weighted avg     0.8819    0.8180    0.8281      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.4023    0.2229    0.2869       157
           2     0.2887    0.5702    0.3833       121
           3     0.6566    0.5285    0.5856       246

    accuracy                         0.4466       524
   macro avg     0.4492    0.4405    0.4186       524
weighted avg     0.4954    0.4466    0.4494       524



In [77]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.491803,0.491803,0.557982,0.464107,0.452290,0.452290,0.496763,0.411747
1,RandomForest,0.997541,0.997541,0.997568,0.997543,0.606870,0.606870,0.589362,0.570661
2,Adaboost,0.609836,0.609836,0.622086,0.566634,0.528626,0.528626,0.484283,0.473907
3,GBM,0.795082,0.795082,0.829448,0.781041,0.547710,0.547710,0.496046,0.495108
4,SVM,0.840164,0.840164,0.845778,0.841137,0.547710,0.547710,0.562625,0.548482
5,KNN,0.629508,0.629508,0.621678,0.617507,0.503817,0.503817,0.487225,0.486648
6,GNB,0.818033,0.818033,0.881900,0.828093,0.446565,0.446565,0.495437,0.449386
